##Install Dependencies

In [0]:
!pip install foolbox
!pip install requests
!pip install keras
!pip install randomstate

##Load a Model to Attack

In [0]:
import foolbox
import keras
import numpy as np
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg19 import VGG19

keras.backend.set_learning_phase(0)
kmodel = ResNet50(weights='imagenet')
#kmodel = VGG19(weights='imagenet')
preprocessing = (np.array([104, 116, 123]), 1) # this is some image manipulation stuff
fmodel = foolbox.models.KerasModel(kmodel, bounds=(0, 255), preprocessing=preprocessing)


##Fetch Image to Target

In [0]:
import requests
from PIL import Image
from io import BytesIO

image_url = "https://cdn.pixabay.com/photo/2016/05/31/21/47/trestle-bridge-1427937_960_720.jpg"
r = requests.get(image_url)
i = Image.open(BytesIO(r.content))
img = i.resize((224, 224))
target_image = np.asarray(img)
label = np.argmax(fmodel.predictions(target_image[:, :, ::-1]))
print(label)

# get source image and label (kitten image example)
#target_image, label = foolbox.utils.imagenet_example()
#print(np.argmax(fmodel.predictions(target_image)))

##Check Target Image Classification

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
from keras.applications.resnet50 import preprocess_input, decode_predictions

# more image processing nonsense
target_rgb = target_image[np.newaxis, :, :, ::-1]
target_rgb = target_rgb.astype(float)
target_predictions = kmodel.predict(preprocess_input(target_rgb.copy()))
target_results = decode_predictions(target_predictions, top=10)

# make note of the label for our attack
#label = target_results[0][0][1]

plt.figure()
plt.subplot(1, 3, 1)
plt.title('Target')
plt.text(0, 255, target_results[0][0][1])
plt.imshow(target_image /255)
plt.axis('off')

##Observe Other Predictions (not required)

In [0]:
print("Predictions:")
for i in target_results[0]:
  print(i[1], "- confidence:", i[2], "\n")

##Apply An Attack to Target Image

In [0]:
#FGSM = Fast Gradient Sign Attack
gradient_attack = foolbox.attacks.FGSM(fmodel)

gradient_adversarial_image = gradient_attack(target_image[:, :, ::-1], label)

##Check our Results

In [0]:
def check_results(target_image, adversarial_image):
  # do some image processing
  adversarial_rgb = adversarial_image[np.newaxis, :, :, ::-1].astype(float)
  real_rgb = target_image[np.newaxis, :, :, ::-1].astype(float)

  # ask the model for predictions for both the real and attack images
  preds_real = kmodel.predict(preprocess_input(real_rgb.copy()))
  preds_adversarial = kmodel.predict(preprocess_input(adversarial_rgb.copy()))
  # get both best guesses
  real_prediction = decode_predictions(preds_real, top=1)[0][0][1]
  adversarial_prediction = decode_predictions(preds_adversarial, top=1)[0][0][1]


  # show it all pretty on the screen
  import matplotlib.pyplot as plt

  plt.figure()

  plt.subplot(1, 3, 1)
  plt.title('Original')
  plt.text(0,255, real_prediction)
  plt.imshow(target_image / 255)  # division by 255 to convert [0, 255] to [0, 1]
  plt.axis('off')

  plt.subplot(1, 3, 2)
  plt.title('Adversarial')
  plt.text(0, 255, adversarial_prediction)
  plt.imshow(adversarial_image[:, :, ::-1] / 255)  # ::-1 to convert BGR to RGB
  plt.axis('off')

  plt.subplot(1, 3, 3)
  plt.title('Difference ')
  difference = adversarial_image[:, :, ::-1] - target_image
  plt.imshow(difference / abs(difference).max() * 0.2 + 0.5)
  plt.axis('off')

  return plt

In [0]:
%matplotlib inline
plt = check_results(target_image, gradient_adversarial_image)
plt.show()

##"Better" Gradient Attacks

###DeepFool

In [0]:
gradient_attack = foolbox.attacks.DeepFoolAttack(fmodel)

gradient_adversarial_image = gradient_attack(target_image[:, :, ::-1], label)

In [0]:
%matplotlib inline
plt = check_results(target_image, gradient_adversarial_image)
plt.show()

###SaliencyMapAttack

In [0]:
gradient_attack = foolbox.attacks.SaliencyMapAttack(fmodel)

gradient_adversarial_image = gradient_attack(target_image[:, :, ::-1], label)

In [0]:
%matplotlib inline
plt = check_results(target_image, gradient_adversarial_image)
plt.show()

##Score-Based Attack

###Single Pixel Attack [SLOW] (see https://arxiv.org/abs/1710.08864)

In [0]:
score_attack = foolbox.attacks.SinglePixelAttack(fmodel)

score_adversarial_image = score_attack(target_image[:, :, ::-1], label)

In [0]:
%matplotlib inline
plt = check_results(target_image, score_adversarial_image)
plt.show()

##Decision-Based Attacks

###Blur

In [0]:
decision_attack = foolbox.attacks.GaussianBlurAttack(fmodel)

decision_adversarial_image = decision_attack(target_image[:, :, ::-1], label)

In [0]:
%matplotlib inline
plt = check_results(target_image, decision_adversarial_image)
plt.show()

###Contrast

In [0]:
decision_attack = foolbox.attacks.ContrastReductionAttack(fmodel)

decision_adversarial_image = decision_attack(target_image[:, :, ::-1], label)

In [0]:
%matplotlib inline
plt = check_results(target_image, decision_adversarial_image)
plt.show()

##Targeted Attacks

###Targeted Gradient Attack

In [0]:
#22 is an eagle
criterion = foolbox.criteria.TargetClass(22)
targeted_attack = foolbox.attacks.LBFGSAttack(fmodel, criterion)
targeted_adversarial_image = targeted_attack(target_image[:, :, ::-1], label)

In [0]:
%matplotlib inline
plt = check_results(target_image, gradient_adversarial_image)
plt.show()

###Targeted Gradient 2

In [0]:
#22 is an eagle
criterion = foolbox.criteria.TargetClass(22)
targeted_attack = foolbox.attacks.FGSM(fmodel, criterion)
targeted_adversarial_image = targeted_attack(target_image[:, :, ::-1], label)



In [0]:
%matplotlib inline
plt = check_results(target_image, gradient_adversarial_image)
plt.show()